<a href="https://colab.research.google.com/github/146790g/pytorch_advanced/blob/master/3_Training_7_training_wo_evaluation_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> **Data Set** :  CT MSD Lung

*   http://medicaldecathlon.com/

> **Coding** **Stye**: Pytorch_TypeA

> **Model**: Simple UNet

> **Section**: Training

> **SubSection**: continued training without evaluation , via bceloss 

In [ ]:
%pwd
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install imgaug==0.4.0
!pip install nibabel==3.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### ランタイムを再実行ボタンを押す

In [ ]:
# パッケージのimport
import torch
import torch.nn as nn
import torch.nn.functional as F
import PIL
import cv2
import tensorflow as tf
print(tf.__version__)

import keras
print(keras.__version__)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset

from pathlib import Path
import glob
import torch
import numpy as np
import imgaug
import imageio
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np
import os
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

import os.path as osp
from PIL import Image

import torch.utils.data as data


# パッケージのimport
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import os
import torch
#import torch.utils.data as data
from pathlib import Path


%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import PIL
import cv2

# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np
import pickle
import datetime
import pytz


import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

import os.path as osp
from PIL import Image

import torch.utils.data as data


# パッケージのimport
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import os
import torch
#import torch.utils.data as data
from pathlib import Path


%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset

2.8.2
2.8.0


In [ ]:
print(cv2.__version__)
print(PIL.__version__)
print(torch.__version__)
print(tf.__version__)
print(keras.__version__)

4.1.2
7.1.2
1.11.0+cu113
2.8.2
2.8.0


### PyTorchの乱数シード固定

In [ ]:
import random
import numpy as np
import torch

def torch_fix_seed(seed=42):
    # Python random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True

torch_fix_seed()

In [ ]:
# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

使用デバイス： cuda:0


In [ ]:
root_dir = Path('/content/gdrive/My Drive/Colab Notebooks/DL/2 Segmentation 03 CT MSD Lung')
data_dir = root_dir/Path('Preprocessed')
dsets_dir = root_dir/Path('dsets_torch')
models_dir = root_dir/Path('models_torch')
cur_dir = root_dir/Path('Pytorch_TypeA_2D Unet')
metrics_dir = cur_dir/Path('metrics')
network_dir = cur_dir/Path('networks')

##set current directory 
##root_dirにセットしないとfrom dsets.dsets import Aが作動しない。
os.chdir(root_dir)
print(os.getcwd())

/content/gdrive/My Drive/Colab Notebooks/DL/2 Segmentation 03 CT MSD Lung


In [ ]:
print(network_dir)

/content/gdrive/My Drive/Colab Notebooks/DL/2 Segmentation 03 CT MSD Lung/Pytorch_TypeA_2D Unet/networks


#Dataloaderからバッチデータを作成

###SegmentationにおけるWeightedRandomSamplerの使い方

In [ ]:
import imgaug
import imageio
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

from torch.utils.data import Dataset
from PIL import Image
from pathlib import Path
import glob
from collections import namedtuple
import os
import random 

# パッケージのimport
import torch
from torchvision import transforms
from PIL import Image, ImageOps, ImageFilter
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset


def change_img_to_label_path(path):
  parts = list(path.parts)
  parts[parts.index("data")] = "masks"
  return Path(*parts)


def make_path_list(root,phase='train'):
  CandidateInfoTuple = namedtuple('CandidateInfoTuple', ['anno_path','img_path'])
  candidateInfo_list=list()
  for img_path in root.glob('{}/*/data/*.npy'.format(phase)):
    anno_path=change_img_to_label_path(img_path)
  
    candidateInfo_list.append(
        CandidateInfoTuple(
            anno_path,
            img_path))
  
  
  return candidateInfo_list


seq = iaa.Sequential([
    iaa.Affine(scale=(0.85, 1.15), # Zoom in or out
               rotate=(-45, 45)),  # Rotate up to 45 degrees
    iaa.ElasticTransformation()  # Random Elastic Deformations
                ])

#segmap = ia.SegmentationMapsOnImage(np.zeros((3, 3, 1), dtype=np.int32), shape=(3, 3, 3), nb_classes=1+5)
#segmap = ia.SegmentationMapsOnImage(np.zeros((3, 3, 1), dtype=np.int32), shape=(3, 3, 3))

#augment_params=seq


def ImageTransform(img,mask):
  random_seed = torch.randint(0, 1000000, (1,)).item()
  imgaug.seed(random_seed)

  mask = SegmentationMapsOnImage(mask, mask.shape)
  img_aug, mask_aug = seq(image=img, segmentation_maps=mask)
  mask_aug = mask_aug.get_arr()
  return img_aug, mask_aug

        
#####################################################################
#  
#   (3) MakeDataset Class
#
#####################################################################
        

import random

        

# with ImageTransform

class MakeDataset(Dataset):

    def __init__(self,file_list,transform=None,phase='train'):
        self.file_list = file_list[phase]
        self.phase = phase
        self.transform = transform

        #random.shuffle(self.file_list)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self,ndx):

        # 1. 画像読み込み
        candidateInfo_tup = self.file_list[ndx]
        img = np.load(candidateInfo_tup.img_path).astype(np.float32)

        # 2. アノテーション画像読み込み
        anno_class_img = np.load(candidateInfo_tup.anno_path).astype(np.int8)
        
        if self.transform:
          img, anno_class_img = self.transform(img, anno_class_img)
          #print('do transform')
          
        #img, anno_class_img = np.expand_dims(img, 0), np.expand_dims(anno_class_img, 0)
        img= np.expand_dims(img, 0)

        return img.astype(np.float32), anno_class_img.astype(np.int8)

In [ ]:
from torch.utils.data import Dataset
from dsets_torch.dsets import ImageTransform
import random
from dsets_torch.dsets import MakeDataset
from dsets_torch.dsets import make_path_list
import torch.utils.data as data


train_list = make_path_list(data_dir,phase="train")
val_list = make_path_list(data_dir,phase="val")

print(len(train_list))
print(len(val_list))

file_list={'train':train_list,'val':val_list}

train_dataset = MakeDataset(file_list=file_list,transform=ImageTransform,phase='train')
val_dataset = MakeDataset(file_list=file_list,phase='val')

print(f"There are {len(train_dataset)} train images and {len(val_dataset)} val images")


17506
1492
There are 17506 train images and 1492 val images


###weight_listの読み込み

In [ ]:
with open(os.path.join(dsets_dir,'weight_list.pickle'),'rb') as f:
  weight_list=pickle.load(f)

print(weight_list[100:500])
print(len(weight_list))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8.094025974025975, 8.094025974025975, 8.094025974025975, 8.094025974025975, 8.094025974025975, 8.094025974025975, 8.094025974025975, 8.094025974025975, 8.094025974025975, 8.094025974025975, 8.094025974025975, 8.094025974025975, 8.094025974025975, 8.094025974025975, 8.094025974025975, 8.094025974025975, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8.094025974025975, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8.094025974025975, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:

batch_size = 20
num_workers = 1

sampler = torch.utils.data.sampler.WeightedRandomSampler(weight_list, len(weight_list))

train_dl = data.DataLoader(train_dataset, batch_size=batch_size,num_workers=num_workers, sampler=sampler)
val_dl = data.DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers,shuffle=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dl, "val": val_dl}


In [ ]:
print(17506//8)
print(1492//8)

2188
186


In [ ]:
print(len(train_dl))
print(len(val_dl))

876
75


In [ ]:
verify_sampler = next(iter(train_dl)) 

In [ ]:
img, label_t = verify_sampler


In [ ]:
data=label_t.numpy()

total=[]
for i in range(len(label_t)):
  num=np.any(data[i])
  total.append(num)

print(sum(total)/len(label_t))

0.4


##Unetのネットワーク構造


In [ ]:
# モデルの定義
import torch
from torch.nn import init
import functools
from torch.autograd import Variable
from torch.optim import lr_scheduler
import numpy as np
from torchvision import models
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function
import torch.utils.data as data
import torch
import os
import pickle 

#####################################################################
#  
#   (1) Network
#
#####################################################################

class DoubleConv(torch.nn.Module):
    """
    Helper Class which implements the intermediate Convolutions
    """
    def __init__(self, in_channels, out_channels):
        
        super().__init__()
        self.step = torch.nn.Sequential(torch.nn.Conv2d(in_channels, out_channels, 3, padding=1),
                                        torch.nn.ReLU(),
                                        torch.nn.Conv2d(out_channels, out_channels, 3, padding=1),
                                        torch.nn.ReLU())
        
    def forward(self, X):
        return self.step(X)


class UNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layer1 = DoubleConv(1, 64)
        self.layer2 = DoubleConv(64, 128)
        self.layer3 = DoubleConv(128, 256)
        self.layer4 = DoubleConv(256, 512)
        
        self.layer5 = DoubleConv(512+256, 256)
        self.layer6 = DoubleConv(256+128, 128)
        self.layer7 = DoubleConv(128+64, 64)
        self.layer8 = torch.nn.Conv2d(64, 1, 1)
        self.layer9 = torch.nn.Sigmoid()
        
        self.maxpool = torch.nn.MaxPool2d(2)
        
    def forward(self, x):
        
        x1 = self.layer1(x)
        x1m = self.maxpool(x1)
        
        x2 = self.layer2(x1m)
        x2m = self.maxpool(x2)
        
        x3 = self.layer3(x2m)
        x3m = self.maxpool(x3)
        
        x4 = self.layer4(x3m)
        
        x5_1 = torch.nn.Upsample(scale_factor=2, mode="bilinear")(x4)
        x5_2 = torch.cat([x5_1, x3], dim=1)
        x5 = self.layer5(x5_2)
        
        x6 = torch.nn.Upsample(scale_factor=2, mode="bilinear")(x5)
        x6 = torch.cat([x6, x2], dim=1)
        x6 = self.layer6(x6)
        
        x7 = torch.nn.Upsample(scale_factor=2, mode="bilinear")(x6)
        x7 = torch.cat([x7, x1], dim=1)
        x7 = self.layer7(x7)
        
        x8 = self.layer8(x7)
        output = self.layer9(x8)
        return output       


In [ ]:
model = UNet()

#学習済みモデルおよび評価指標の保存用の関数の作成

#train_model関数の作成

In [ ]:
print(device)

cuda:0


In [ ]:
def save_model(network, epoch_ndx):
  save_filename = 'model_%s.pth' % (epoch_ndx)
  save_path = os.path.join(network_dir, save_filename)
  torch.save(network.cpu().state_dict(), save_path)


def train_model(model,dataloaders_dict,epoch_start,epoch_end,lr):

  now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
  print('start time:',now)

  t_epoch_start = time.time()
  #optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)

  optimizer = optim.Adam(model.parameters(),lr=lr)
  #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

  num_epochs=epoch_end-epoch_start+1

  for epoch_ndx in range(epoch_start,epoch_end+1):
   
    print('-----------------------------------------------------------------------------------------')
    now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
    print('Epoch:{} 【From Epoch {}:To Epoch {}】:Now {}'.format(epoch_ndx,epoch_start,epoch_end,now))
    print('-----------------------------------------------------------------------------------------')

    model.to(device)
    for phase in ['train', 'val']:

      t_epoch_start = time.time()
      iteration = 1

      if phase == 'train':
        model.train()  # モデルを訓練モードに
        #print('-------------')
        print('（train）')
        #print('-------------')
      else:
        model.eval()   # モデルを検証モードに
        #print('-------------')
        print('（val）')
        #print('-------------')
      
      epoch_loss = 0.0  # epochの損失和

      # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
      if (epoch_ndx == 0) and (phase == 'train'):
        continue

      #for batch_ndx, batch_tup in enumerate(tqdm(dataloaders_dict[phase])):
      for batch_ndx, batch_tup in enumerate(dataloaders_dict[phase]):

        t_iter_start = time.time()
        input_t, label_t = batch_tup
        input_g=input_t.to(device)
        label_g=label_t.to(device)
        del input_t,label_t,batch_tup

        # optimizerを初期化
        optimizer.zero_grad()

      # 順伝搬（forward）計算
        with torch.set_grad_enabled(phase == 'train'):
        
          outputs = model(input_g)
          loss_g = F.binary_cross_entropy_with_logits(outputs.view(-1).float(), label_g.view(-1).float())
          #loss_g = torch.nn.BCEWithLogitsLoss(outputs.view(-1).float(), label_g.view(-1).float())
          epoch_loss += loss_g.item() * input_g.size(0) 

          # 訓練時はバックプロパゲーション
          if phase == 'train':
            loss_g.backward()
            optimizer.step()

      #phaseごとのlossとaccuracy
      t_epoch_finish = time.time()
      #print('---------------------------------------------------------------------------------------')
      print('Duration: {:.4f} sec. '.format(t_epoch_finish - t_epoch_start))
      #print('---------------------------------------------------------------------------------------')
      epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
      print('{} Loss: {:.4f}'.format(phase, epoch_loss))

    #epochごとに周期的にモデルを保存
    if (epoch_ndx % save_freq ==0) & (epoch_ndx>0):
      print('saving the model at the end of epoch %d' % epoch_ndx)
      save_model(model,epoch_ndx)
 


###①初回学習の実行

RuntimeWarning: Mean of empty sliceが出たときには、batch_sizeを変える

In [ ]:
# 学習・検証を実行する
epoch_start=4
save_freq=3
epoch_end=save_freq*2
lr = (1e-04)*0.5

train_model(model,dataloaders_dict,epoch_start,epoch_end,lr)    


start time: 2022-06-09 11:16:51.817831+09:00
-----------------------------------------------------------------------------------------
Epoch:4 【From Epoch 4:To Epoch 6】:Now 2022-06-09 11:16:51.818408+09:00
-----------------------------------------------------------------------------------------
（train）
Duration: 1048.9961 sec. 
train Loss: 0.7019
（val）
Duration: 20.5216 sec. 
val Loss: 0.6931
-----------------------------------------------------------------------------------------
Epoch:5 【From Epoch 4:To Epoch 6】:Now 2022-06-09 11:34:41.351793+09:00
-----------------------------------------------------------------------------------------
（train）
Duration: 743.1975 sec. 
train Loss: 0.6931
（val）
Duration: 20.5283 sec. 
val Loss: 0.6931
-----------------------------------------------------------------------------------------
Epoch:6 【From Epoch 4:To Epoch 6】:Now 2022-06-09 11:47:25.083403+09:00
-----------------------------------------------------------------------------------------
（tr

In [ ]:
epoch_ndx=6

from models_torch.models import UNet
model = UNet()

load_filename = 'model_%s.pth' % (epoch_ndx)
load_path = os.path.join(network_dir, load_filename)

# PyTorchのネットワークパラメータのロード
model.load_state_dict(torch.load(load_path))


<All keys matched successfully>

In [61]:
# 学習・検証を実行する
epoch_start=7
save_freq=5
epoch_end=save_freq*20
lr = (1e-04)*0.5

train_model(model,dataloaders_dict,epoch_start,epoch_end,lr)  

start time: 2022-06-09 12:24:33.223822+09:00
-----------------------------------------------------------------------------------------
Epoch:7 【From Epoch 7:To Epoch 100】:Now 2022-06-09 12:24:33.224923+09:00
-----------------------------------------------------------------------------------------
（train）
Duration: 844.0199 sec. 
train Loss: 0.6931
（val）
Duration: 20.5058 sec. 
val Loss: 0.6931
-----------------------------------------------------------------------------------------
Epoch:8 【From Epoch 7:To Epoch 100】:Now 2022-06-09 12:38:57.766087+09:00
-----------------------------------------------------------------------------------------
（train）
Duration: 772.5893 sec. 
train Loss: 0.6931
（val）
Duration: 20.5303 sec. 
val Loss: 0.6931
-----------------------------------------------------------------------------------------
Epoch:9 【From Epoch 7:To Epoch 100】:Now 2022-06-09 12:52:10.890460+09:00
----------------------------------------------------------------------------------------

KeyboardInterrupt: ignored

In [62]:
class DiceScore(torch.nn.Module):
    """
    class to compute the Dice Loss
    """
    def __init__(self):
        super().__init__()

    def forward(self, pred, mask):
                
        #flatten label and prediction tensors
        pred = torch.flatten(pred)
        mask = torch.flatten(mask)
        
        counter = (pred * mask).sum()  # Counter       
        denum = pred.sum() + mask.sum()  # denominator
        dice = (2*counter)/denum
        
        return dice


In [63]:
preds = []
labels = []

for slice, label in tqdm(val_dataset):
    slice = torch.tensor(slice).float().to(device).unsqueeze(0)
    model.to(device)
    with torch.no_grad():
        pred = model(slice)
    preds.append(pred.cpu().numpy())
    labels.append(label)
    
preds = np.array(preds)
labels = np.array(labels)

100%|██████████| 1492/1492 [00:37<00:00, 40.26it/s]


In [64]:
dice_score = DiceScore()(torch.from_numpy(preds), torch.from_numpy(labels).unsqueeze(0).float())
print(f"The Val Dice Score is: {dice_score}")

The Val Dice Score is: 0.0


#END